# Custom CNN in Keras with GPU

- I built this custom CNN using Keras
- The model was trained on AMD GPU with PlaidML backend. I did not use TensorFlow since i do not have an Nvidia GPU, but that can be changed by setting `environ["KERAS_BACKEND"] = "tensorflow"`
- This model can classify dogs and cats
- The training dataset consists of 8000 images for both cats and dogs, total of 16000 images
- The validation dataset consists of 3500 images for both cats and dogs, total of 7000 images
- The testing dataset consists of 1000 images for both cats and dogs, total of 2000 images
- In total we have 25000 images

## Import all libraries and set backend to PlaidML

To use PlaidML as backend follow these steps:
- activate project environment
- `plaidml-setup`
- `Enable experimental device support? (y,n)[n]:n`
- `Please choose a default device: 3` most probably 1 would be your CPU, 2 would be the integrated GPU of your CPU and 3 would be your dedicated GPU
- `Save settings to /Users/veersingh/.plaidml? (y,n)[y]:y`

In [ ]:
# import libraries and set backend to plaidML
from os import environ
environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.preprocessing.image import ImageDataGenerator
import time
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import json
import PIL

## Preprocessing the data

- First we need to rescale the pixel values. Currently the pixel values are in the range [0,255] since we are using 8 bit color or 2^8 values per pixel. We can just divide each pixel value with 255 and that will rescale all pixels to the range [0,1]. This reduces computation.
- Next we need to label and resize the image since the model accepts images of the fixed size 100x100x3 where the image is 100 pixels in length and width and has 3 channels RGB. We can do this automatically by using ImageDataGenerator which has the method flow_from_director.

In [ ]:
# Specify the paths
train_dataset_path = '/Users/veersingh/Desktop/cats_dogs/train'
valid_dataset_path = '/Users/veersingh/Desktop/cats_dogs/valid'

In [ ]:
# Rescale
train = ImageDataGenerator(rescale=1 / 255)
valid = ImageDataGenerator(rescale=1 / 255)

# Define train and valid dataset
# This method is useful since our images are in their respective class folders

# seed = random number to maintain same output everytime
# directory = path of train or valid dataset
# target_size = resize the input image to required shape
# batch_size = default batch size of 32
# class_mode = categorical for multiclass
# color_mode = depending on number of channels required by model, grayscale for 1 channel, rgb for 3 channel

train_dataset = train.flow_from_directory(seed=1,
                                          directory=train_dataset_path,
                                          target_size=(100, 100),
                                          batch_size=32,
                                          class_mode = 'binary',
                                          color_mode='rgb')

valid_dataset = valid.flow_from_directory(seed=1,
                                         directory=valid_dataset_path,
                                         target_size=(100, 100),
                                         batch_size=32,
                                         class_mode = 'binary',
                                         color_mode='rgb')

# print labels
print('\n')
print(f'Label for cat -> {train_dataset.class_indices["cat"]}')
print(f'Label for dog -> {train_dataset.class_indices["dog"]}')

## Create the Model

In [ ]:
# Model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3, 3),activation='relu',input_shape=(100, 100, 3)))

# Maxpool layer
model.add(MaxPooling2D(2, 2))

# Convolutional layer 2and maxpool layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))

# Maxpool layer
model.add(MaxPooling2D(2, 2))

# Convolutional layer 3
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))

# Maxpool layer
model.add(keras.layers.MaxPool2D(2, 2))

# Convolutional layer 4
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))

# Maxpool layer
model.add(keras.layers.MaxPooling2D(2, 2))

# Flatten to 1D
model.add(keras.layers.Flatten())

# Fully connected layer with 512 neurons
model.add(keras.layers.Dense(512, activation='relu'))

# Output layer with single neuron which gives 0 for cat and 1 for dog
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

## Train and Save the Model

In [ ]:
# Train the model

# start the timer
start_time = time.time()

# generator = input training data created using ImageDataGenerator
# steps_per_epoch = number of total training images/batch size = 16000/32 = 500
# epochs = number of times we go through the entire training dataset
# validation_data = validation dataset
# verbose = it will print updates on every epoch

history = model.fit_generator(generator=train_dataset,
                              steps_per_epoch=500,
                              epochs=30,
                              validation_data=valid_dataset,
                              verbose=2)

# save the model
model.save('cats_dogs_trained_model.h5')

# stop the timer
time_elapsed = round(time.time()-start_time, 2)

print(f'\n\nThe model took {time_elapsed} seconds' )